# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [2]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU

import pandas as pd

import matplotlib.pyplot as plt

import sqlite3

import numpy as np

augment_df = pd.read_csv("https://bit.ly/3BPcobU",encoding = 'unicode_escape')
print(augment_df.shape)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(52870, 20)


In [3]:
# checking missing value counts
augment_df.isnull().sum()

company_permalink             1
company_name                  1
company_category_code       643
company_country_code          1
company_state_code          492
company_region                1
company_city                533
investor_permalink            2
investor_name                 2
investor_category_code    50427
investor_country_code     12001
investor_state_code       16809
investor_region               2
investor_city             12480
funding_round_type            3
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
raised_amount_usd          3599
dtype: int64

In [4]:
# checking Each column's memory footprint.
chrunch_iter = pd.read_csv("https://bit.ly/3BPcobU",chunksize=5000,encoding = 'unicode_escape')
for chunkx in chrunch_iter:  
  print(chunkx.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

## Findings

The columns that can be dropped is investor_category_code since it have a lot of null values

In [5]:
# dropping investor_category_code and rows where raised amounts is null 

for chunkx in chrunch_iter:
    chunkx = chunkx.dropna(subset=['raised_amount_usd'])
    chunkx.drop(['investor_category_code'], axis=1)
print(chunkx.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2870 entries, 50000 to 52869
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       2870 non-null   object 
 1   company_name            2870 non-null   object 
 2   company_category_code   2860 non-null   object 
 3   company_country_code    2870 non-null   object 
 4   company_state_code      2841 non-null   object 
 5   company_region          2870 non-null   object 
 6   company_city            2825 non-null   object 
 7   investor_permalink      2870 non-null   object 
 8   investor_name           2870 non-null   object 
 9   investor_category_code  0 non-null      float64
 10  investor_country_code   0 non-null      float64
 11  investor_state_code     0 non-null      float64
 12  investor_region         2870 non-null   object 
 13  investor_city           0 non-null      float64
 14  funding_round_type      2870 non-nu

In [6]:
## dropping investor_category_code and rows where raised amounts is null because they are unnecesary
for chunkx in augment_df:
  chunkx = augment_df.dropna(subset=['raised_amount_usd'])
  chunk = chunkx.drop(['investor_category_code'], axis=1) 
print(chunk.head())

     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA                 OH               Columbus   
4                  USA                 OH              Cleveland   

    company_city          investor_permalink      investor_name  \
0  San Francisco  /company/1-800-flowers-com  1-800-FLOWERS.COM   
1  Mountain View        /company/10xelerator        10Xelerator   
2            NaN       

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [7]:
# Identifying the data types of each columns.

print(chunk.dtypes)

company_permalink         object
company_name              object
company_category_code     object
company_country_code      object
company_state_code        object
company_region            object
company_city              object
investor_permalink        object
investor_name             object
investor_country_code     object
investor_state_code       object
investor_region           object
investor_city             object
funding_round_type        object
funded_at                 object
funded_month              object
funded_quarter            object
funded_year              float64
raised_amount_usd        float64
dtype: object


#Identify the numeric columns we can represent using more space efficient types. For text columns:

In [8]:
#numeric columns identified phisically
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','complex']

chunk_numeric = chunk.select_dtypes(include=numerics)
print("Columns that have numeric types are:",chunk_numeric.columns.tolist())

Columns that have numeric types are: ['funded_year', 'raised_amount_usd']


In [9]:
# Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type
unique_cols = [] 
for col in chunk.select_dtypes(include=['object']).columns:
  if chunk[col].nunique()/len(chunk)< 0.5:
    unique_cols.append(col)
unique_cols = list(set(unique_cols))
print(unique_cols)

['company_category_code', 'funded_quarter', 'funding_round_type', 'investor_city', 'investor_state_code', 'company_region', 'company_name', 'company_permalink', 'funded_at', 'funded_month', 'company_state_code', 'company_country_code', 'investor_country_code', 'investor_region', 'investor_permalink', 'company_city', 'investor_name']


In [10]:
# See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
import pandas as pd

to_drop = [] # list of columns to drop
low_unique_cols = [] # list of columns with low unique values

augment_df = pd.read_csv("https://bit.ly/3BPcobU",encoding = 'unicode_escape', chunksize = 100)
memory_usage = 0 

for chunk in augment_df:
    # leaving out the identified columns
    chunk = chunk.drop(to_drop, axis=1)
  
    chunk[chunk.select_dtypes(include=['float']).columns] = chunk.select_dtypes(include=['float']).apply(pd.to_numeric, downcast='float')
   
    chunk[chunk.select_dtypes(include=['int']).columns] = chunk.select_dtypes(include=['int']).apply(pd.to_numeric, downcast='integer')
    
    for col in low_unique_cols:
        if col in chunk.columns:
            chunk[col] = chunk[col].astype('category')
    
    chunk_memory = chunk.memory_usage(deep=True).sum() / (1024 ** 2)
    print("memory in use for the chunk is:", chunk_memory, "MB")
    
    memory_usage += chunk_memory

print("memory usage:", memory_usage, "MB")

memory in use for the chunk is: 0.11008930206298828 MB
memory in use for the chunk is: 0.11035346984863281 MB
memory in use for the chunk is: 0.10944652557373047 MB
memory in use for the chunk is: 0.11162567138671875 MB
memory in use for the chunk is: 0.11048221588134766 MB
memory in use for the chunk is: 0.10946083068847656 MB
memory in use for the chunk is: 0.11212158203125 MB
memory in use for the chunk is: 0.10920524597167969 MB
memory in use for the chunk is: 0.10854244232177734 MB
memory in use for the chunk is: 0.11170101165771484 MB
memory in use for the chunk is: 0.11116504669189453 MB
memory in use for the chunk is: 0.10895824432373047 MB
memory in use for the chunk is: 0.10843086242675781 MB
memory in use for the chunk is: 0.10976028442382812 MB
memory in use for the chunk is: 0.11037826538085938 MB
memory in use for the chunk is: 0.11044120788574219 MB
memory in use for the chunk is: 0.11071014404296875 MB
memory in use for the chunk is: 0.11048412322998047 MB
memory in use

In [11]:
# Making changes to ensure overall memory stays under 10 megabytes.

import pandas as pd

# Read the CSV file in chunks of 1000 rows
augment_df = pd.read_csv("https://bit.ly/3BPcobU", chunksize=1000, encoding='unicode_escape')

# Initialize an empty list to store the records
records = []

# Keep track of the total memory usage
memory_usage = 0

# Iterate over the chunks
for chunk in augment_df:
    # Select only the columns with object dtype
    text_cols = chunk.select_dtypes(include='object')
    
    # Sample 1000 rows from the text columns
    samples = text_cols.sample(1000)
    
    # Append the samples to the records list
    records.append(samples)
    
    # Update the total memory usage
    memory_usage += samples.memory_usage(deep=True).sum()/2**20
    
    # If the total memory usage exceeds 10 MB, break the loop
    if memory_usage > 10:
        break

# Concatenate the records into a single DataFrame
final_df = pd.concat(records)

# Print the total memory usage
print(f'Overall consumption is {memory_usage} Mbs')


Overall consumption is 10.96226692199707 Mbs


#Finding
The memory usage is now 10.96mb down from 56mb.

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [19]:
# Create and connect to a new SQLite database file.
conn = sqlite3.connect("funding.db")
augment_df = pd.read_csv("https://bit.ly/3BPcobU", chunksize=1000,encoding = 'unicode_escape')
for chunk in augment_df:
    chunk.to_sql("funding", conn, if_exists='append', index=False)
    df = pd.read_sql('PRAGMA table_info(funding);', conn)
print(df)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

In [20]:
#
# Create a cursor object
cursor = conn.cursor()

# Query the table
cursor.execute("PRAGMA table_info(funding)")

# Get the column names and types
columns = cursor.fetchall()
for column in columns:
  print(column[1], column[2])

company_permalink TEXT
company_name TEXT
company_category_code TEXT
company_country_code TEXT
company_state_code TEXT
company_region TEXT
company_city TEXT
investor_permalink TEXT
investor_name TEXT
investor_category_code TEXT
investor_country_code TEXT
investor_state_code TEXT
investor_region TEXT
investor_city TEXT
funding_round_type TEXT
funded_at TEXT
funded_month TEXT
funded_quarter TEXT
funded_year INTEGER
raised_amount_usd REAL


In [21]:
# Query the table and make sure the data types match up to what you had in mind for each column.
outcome_df = pd.read_sql('PRAGMA table_info(funding);', conn)
print(outcome_df)

    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     TEXT        0       None   0
10   10   investor_country_code     TEXT        0       None   0
11   11     investor_state_code     TEXT        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     TEXT        0       None   0
14   14      funding_roun

In [22]:
df = pd.read_sql("SELECT * FROM funding LIMIT 5", conn)
df

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,None,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000.0
2,/company/utap,uTaP,messaging,USA,None,United States - Other,None,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000.0


# conclusion. 
data is stored in our desired formart in an sql format

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [23]:
# Your code goes here
# What proportion of the total amount of funds did the top 10% raise? What about the top 1%? 
# Compare these values to the proportions the bottom 10% and bottom 1% raised.

df = pd.read_sql("SELECT * FROM funding", conn)
group_df = df.groupby("company_name").sum()
sort_df= group_df.sort_values("raised_amount_usd", ascending=False)
total_funds = sort_df["raised_amount_usd"].sum()

# proportion of funds raised by the top 10%
top_10 = sort_df.iloc[:int(len(sort_df) * 0.1)]["raised_amount_usd"].sum()
top_10_proportion = top_10 / total_funds

# proportion of funds raised by the top 1%
top_1 = sort_df.iloc[:int(len(sort_df) * 0.01)]["raised_amount_usd"].sum()
top_1_proportion = top_1 / total_funds

# Output of the expectation

print("Proportion of total funds raised by top 10% is:", top_10_proportion)
print("Proportion of total funds raised by top 1%:", top_1_proportion)

Proportion of total funds raised by top 10% is: 0.6712772709906496
Proportion of total funds raised by top 1%: 0.26217737281766545


In [25]:
# comparing values of top 10%, 1% and values of bottom 10% and 1%
bottom_10 = sort_df.iloc[-int(len(sort_df) * 0.1):]["raised_amount_usd"].sum()
bottom_10_proportion = bottom_10 / total_funds

bottom_1 = sort_df.iloc[-int(len(sort_df) * 0.01):]["raised_amount_usd"].sum()
bottom_1_proportion = bottom_1 / total_funds

# output of expectation

print("bottom 10%'s proportion of funds raised:", bottom_10)
print("bottom 1%'s proportion of funds raised:", bottom_1_proportion)

bottom 10%'s proportion of funds raised: 0.0
bottom 1%'s proportion of funds raised: 0.0


In [27]:
# To check which category of companies attracted the most investment, do the following;
most_ivestments_df = df.groupby("company_category_code").sum()
most_ivestments_df_sorted = most_ivestments_df.sort_values("raised_amount_usd", ascending=False)
category_most_invested = most_ivestments_df_sorted.index[0]

# output the results
print("Company category that attracted the most investments is:", category_most_invested)

Company category that attracted the most investments is: biotech


In [28]:
# To get the investor that contributed alot across all startups
most_ivestments_df = df.groupby("investor_name").sum()
most_ivestments_df_sorted = most_ivestments_df.sort_values("raised_amount_usd", ascending=False)
investor_most_contributor = most_ivestments_df_sorted.index[0]

# Output the findings
print("most money contributed came from:", investor_most_contributor)


most money contributed came from: Kleiner Perkins Caufield & Byers


In [30]:
# to get investors that contributed the most per start up
most_ivestments_df = df.groupby(["company_name","investor_name"]).agg({"raised_amount_usd": "sum"})
most_ivestments_df_sorted = most_ivestments_df.sort_values("raised_amount_usd", ascending=False)
first_10_investors = most_ivestments_df_sorted.head(10)
print(first_10_investors)

                                       raised_amount_usd
company_name investor_name                              
Clearwire    Time Warner                    5.620000e+09
             Comcast                        5.620000e+09
             Intel                          5.620000e+09
             BrightHouse                    4.700000e+09
             Google                         3.200000e+09
sigmacare    Marlin Equity Partners         2.600000e+09
Clearwire    Sprint Nextel                  2.500000e+09
             Eagle River Holdings           2.420000e+09
Facebook     Digital Sky Technologies       1.700000e+09
             Goldman Sachs                  1.500000e+09


In [31]:
# Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

most_ivestments_df = df.groupby("funding_round_type").count()
most_ivestments_df_sorted = most_ivestments_df.sort_values("company_name", ascending=False)
most_popular_round = most_ivestments_df_sorted.index[0]
least_popular_round = most_ivestments_df_sorted.index[-1]

# output the finding
print("The most popular funding round is:", most_popular_round)
print("The least popular funding round is:", least_popular_round)

# It is a good practice to close the connection after you are done using it.
conn.close()

The most popular funding round is: series-a
The least popular funding round is: crowdfunding
